In [ ]:
# --- Imports & Setup ---
import pandas as pd
import matplotlib.pyplot as plt
from shapely import affinity
from shapely.geometry import Polygon
import sys
import os

# Tambahkan folder src ke path agar bisa import geometry.py
sys.path.append(os.path.abspath('../'))
from src.geometry import get_tree_polygon

# Fungsi visualisasi helper
def plot_puzzle(df_puzzle, title="Tree Packing Visualization"):
    fig, ax = plt.subplots(figsize=(10, 10))
    base_tree = get_tree_polygon()
    
    polys = []
    for _, row in df_puzzle.iterrows():
        # Parse koordinat (hapus 's')
        x = float(str(row['x']).replace('s', ''))
        y = float(str(row['y']).replace('s', ''))
        deg = float(str(row['deg']).replace('s', ''))
        
        # Transformasi
        p = affinity.rotate(base_tree, deg, origin=(0,0))
        p = affinity.translate(p, x, y)
        polys.append(p)
        
        # Gambar Pohon
        x_p, y_p = p.exterior.xy
        ax.fill(x_p, y_p, alpha=0.6, fc='green', ec='black')
        
        # Tulis ID pohon di tengahnya
        ax.text(x, y, row['id'].split('_')[1], ha='center', va='center', color='white', fontsize=8)

    # Hitung Bounding Box
    min_x = min(p.bounds[0] for p in polys)
    max_x = max(p.bounds[2] for p in polys)
    min_y = min(p.bounds[1] for p in polys)
    max_y = max(p.bounds[3] for p in polys)
    
    width = max_x - min_x
    height = max_y - min_y
    side = max(width, height)
    
    # Gambar Kotak Batas
    rect = plt.Rectangle((min_x, min_y), width, height, linewidth=2, edgecolor='red', facecolor='none', linestyle='--')
    ax.add_patch(rect)
    
    ax.set_aspect('equal')
    ax.set_title(f"{title}\nBounding Box: {width:.2f} x {height:.2f} (Score: {side**2/len(df_puzzle):.4f})")
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
submission_path = "../submission.csv" 
sample_path = "../data/raw/sample_submission.csv"

if os.path.exists(submission_path):
    print("Memvisualisasikan HASIL SOLUSI Anda...")
    df = pd.read_csv(submission_path)
else:
    print("Memvisualisasikan SAMPLE AWAL (belum dioptimasi)...")
    df = pd.read_csv(sample_path)

# Ambil satu puzzle saja (misal puzzle dengan 5 pohon)
puzzle_id = "005" 
subset = df[df['id'].str.startswith(puzzle_id)]

plot_puzzle(subset, title=f"Puzzle {puzzle_id}")